<a href="https://colab.research.google.com/github/yowsitian/webScrapping1/blob/main/Web_Scraping_Beautiful_Soap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install selenium

In [ ]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import time
import json

companyList = [
    {'id': 'citylink', 'link':'City-link%20Express', 'listLinks':[],'contents':''},
    {'id': 'poslaju','link':'Pos%20Laju', 'listLinks':[],'contents':''},
    {'id': 'gdex', 'link':'GDEX', 'listLinks':[],'contents':''},
    {'id': 'jnt', 'link':'J%26T%20express', 'listLinks':[],'contents':''},
    {'id': 'dhl', 'link':'DHL', 'listLinks':[],'contents':''},
]

urlList = [
    {'id': 'theStar', 'link': 'https://www.thestar.com.my/search/?q='},
    {'id': 'borneoPost','link': 'https://www.theborneopost.com/search_gcse/?q='},
    {'id': 'newStraitsTimes', 'link': 'https://www.nst.com.my/search?keywords='},
]

In [ ]:
def checkValidity(url):
    r = requests.get(url)
    if r.status_code == 200:
        return r
    return False

In [ ]:
def retrieveArticleLinks(inputList):
    for i in inputList:
        index = 0
        while len(i['listLinks'])  < 3 and index != len(urlList):
            url = urlList[index]['link'] + i['link']
            r = checkValidity(url)
            if r:
                contentNews = []
                soup1 = BeautifulSoup(r.content, "html.parser")
                # News identification
                if urlList[index]['id'] == 'theStar':
                    contentNews = soup1.find_all('h2', class_='f18')
                elif urlList[index]['id'] == 'borneoPost':
                    contentNews = soup1.find_all('div', class_='gs-title')
                else:
                    contentNews = soup1.find_all('div', class_='article-teaser')
                for n in np.arange(0,len(contentNews)):
                    if n == 3:
                        break
                    i['listLinks'].append(contentNews[n].find('a')['href'])
                index += 1
    return inputList

In [ ]:
print(retrieveArticleLinks(companyList))

[{'id': 'citylink', 'link': 'City-link%20Express', 'listLinks': ['https://www.thestar.com.my/business/business-news/2020/10/27/freeze-on-courier-licences-for-two-years', 'https://www.thestar.com.my/tech/tech-news/2020/10/26/mcmc-freezes-applications-for-new-courier-licenses-until-2022', 'https://www.thestar.com.my/metro/metro-news/2018/09/03/bargains-galore-for-eshoppers-fiveday-online-sale-beginning-today-offers-special-deals-on-products-b'], 'contents': ''}, {'id': 'poslaju', 'link': 'Pos%20Laju', 'listLinks': ['https://www.thestar.com.my/opinion/letters/2021/04/17/where-is-my-parcel-mr-postman', 'https://www.thestar.com.my/news/nation/2021/04/10/serve-your-two-years-reckless-biker-told', 'https://www.thestar.com.my/news/nation/2021/02/05/retiree-duped-out-of-nearly-rm1mil-in-macau-scam'], 'contents': ''}, {'id': 'gdex', 'link': 'GDEX', 'listLinks': ['https://www.thestar.com.my/business/business-news/2021/05/01/surge-in-demand-for-logistics-e-commerce', 'https://www.thestar.com.my/bu

In [ ]:
def scrapeSingleWeb(url):
    r = checkValidity(url)
    result = ''
    if r:
        article = BeautifulSoup(r.content, "html.parser")
        arr = []
        if "www.thestar.com" in url:
          body = article.find('div',attrs={"id": "story-body"})   
          arr = body.findAll('p')
        for j in arr:
            result += str(j.get_text())  
    return result

In [ ]:
companyList = retrieveArticleLinks(companyList)
for i in companyList:
    index = 0
    #pass article link by link
    while index < 3:
        i['contents'] += str(scrapeSingleWeb(i['listLinks'][index]))
        index += 1
print(companyList)

[{'id': 'citylink', 'link': 'City-link%20Express', 'listLinks': ['https://www.thestar.com.my/business/business-news/2020/10/27/freeze-on-courier-licences-for-two-years', 'https://www.thestar.com.my/tech/tech-news/2020/10/26/mcmc-freezes-applications-for-new-courier-licenses-until-2022', 'https://www.thestar.com.my/metro/metro-news/2018/09/03/bargains-galore-for-eshoppers-fiveday-online-sale-beginning-today-offers-special-deals-on-products-b'], 'contents': 'PETALING JAYA: In a move to strengthen the local postal and courier industry that is rife with challenges, the Malaysian Communications and Multimedia Commission (MCMC) will not issue any new courier licences for two years.Additionally, a new action plan will be drafted by the National Postal and Courier Laboratory Laboratory (NPCIL) to propose measures to support the strategic development of the new postal and courier industry.In a press release yesterday, MCMC said the moratorium was implemented to allow the government, along with 

In [ ]:
df = pd.DataFrame.from_dict(companyList)
from google.colab import  drive
drive.mount('/drive')
df.to_csv('/drive/My Drive/Year 2/name_csv_file.csv')